In [9]:
import bokeh
import json
import pandas as pd
import numpy as np
import requests
from bokeh.plotting import figure, output_file, show, output_notebook
from bokeh.models import NumeralTickFormatter
from bokeh.io import show
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, CustomJS, Select
from bokeh.plotting import figure
from ARCTIC import hdf5_interface
from ARCTIC import nrel_api_interface

To call a number of different locations all at once, just call using a dictionary with the location names as the keys and have nested lat and lon keys therein, and then the lat and lon. Save the figures as the dictionary keys+ the .format construction that includes the tilt plot name or w/e. Also allow option to pass a list of tilts to make it customizable.

In [6]:
location_dataframe = pd.DataFrame(columns=['location','latitude','longitude'])
location_dataframe['location']=['Ambler-Shungnak-Kobuk','Anchorage','Bethel','Chickaloon',
                       'Deering','Denali Park','Fairbanks','Fort Yukon',
                       'Galena-Koyukuk-Ruby', 'Homer','Naknek','Noatak',
                       'Noorvik','Soldotna','Valdez','Wasilla-Palmer']

location_dataframe['latitude']=[66.995834, 61.193625, 60.794938, 61.823570,
                       66.069413, 63.537277, 64.838033, 66.571563,
                       64.782991, 59.652521, 58.728349, 67.570921,
                       66.836039, 60.486370, 61.128663, 61.582242]


location_dataframe['longitude']=[ -157.377096, -149.694974, -161.770716, -148.450442,
                        -162.766760,  -150.985453, -147.668970,  -145.250173,
                        -156.744933, -151.536496, -157.017444, -162.967490,
                         -161.041913, -151.060702, -146.353366, -149.441001]


location_dataframe

,location,latitude,longitude
0,Ambler-Shungnak-Kobuk,66.995834,-157.377096
1,Anchorage,61.193625,-149.694974
2,Bethel,60.794938,-161.770716
3,Chickaloon,61.823570,-148.450442
4,Deering,66.069413,-162.766760
5,Denali Park,63.537277,-150.985453
6,Fairbanks,64.838033,-147.668970
7,Fort Yukon,66.571563,-145.250173
8,Galena-Koyukuk-Ruby,64.782991,-156.744933
9,Homer,59.652521,-151.536496


In [37]:
def tilt_angle_plot_generation(location_dataframe): 
    """This function takes in a dataframe that contains latitudes and longitudes for a number of 
    locations and generates interactive Bokeh plots showing the variation of monthly production 
    with changing tilt angles."""
    #The below list is sufficiently granular to cover most situations.
    tilt_list = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90]

    #Walk through each row in the location dataframe, calling PVwatts and plotting results.
    for j in range(len(location_dataframe.index)):
        print("Data for " + str(location_dataframe['location'][j]) + " is being calculated")
        nrel_long_tilt = []
        for i in range(len(tilt_list)):
            list_parameters = {"formt": 'JSON', "api_key": "spJFj2l5ghY5jwk7dNfVYs3JHbpR6BOGHQNO8Y9Z", "system_capacity": 1, "module_type": 0, "losses": 14.08,
                      "array_type": 0, "tilt": tilt_list[i], "azimuth": 180, "lat": location_dataframe['latitude'][j], "lon": location_dataframe['longitude'][j], "dataset": 'tmy3'}
            json_response = requests.get("https://developer.nrel.gov/api/pvwatts/v6", params = list_parameters).json()
            new_dataframe = pd.DataFrame(data = json_response['outputs'])
            nrel_long_tilt.append(new_dataframe)
        tilt_response_dataframe = pd.DataFrame(columns = tilt_list)
        for i, tilt in enumerate(tilt_list):
            tilt_response_dataframe[tilt] = nrel_long_tilt[i]['ac_monthly']

        #The below is all of the data for the plotting components.
        #This adjusts the name of the saved file, so it's specific to each location.
        output_file("{}_monthly_production_varying_tilts.html".format(location_dataframe['location'][j]))
        #Set up a month proxy
        x = np.arange(1,13)

        #Tell the plot where to look for the data. The extra specifications of y values
        #enable the plot to be interactive.
        source = ColumnDataSource(data=dict(x=x, y=tilt_response_dataframe[5],
                                            tilt_5_degrees=tilt_response_dataframe[5], tilt_10_degrees=tilt_response_dataframe[10],
                                            tilt_15_degrees=tilt_response_dataframe[15], tilt_20_degrees=tilt_response_dataframe[20],
                                            tilt_25_degrees=tilt_response_dataframe[25], tilt_30_degrees=tilt_response_dataframe[30],
                                            tilt_35_degrees=tilt_response_dataframe[35], tilt_40_degrees=tilt_response_dataframe[40],
                                            tilt_45_degrees=tilt_response_dataframe[45], tilt_50_degrees=tilt_response_dataframe[50],
                                            tilt_55_degrees=tilt_response_dataframe[55], tilt_60_degrees=tilt_response_dataframe[60],
                                            tilt_65_degrees=tilt_response_dataframe[65], tilt_70_degrees=tilt_response_dataframe[70],
                                            tilt_75_degrees=tilt_response_dataframe[75], tilt_80_degrees=tilt_response_dataframe[80],
                                            tilt_85_degrees=tilt_response_dataframe[85], tilt_90_degrees=tilt_response_dataframe[90],
                                           ))
        #Plot specifications
        plot = figure(x_axis_label='Month', y_axis_label='Normalized Monthly Production (kWh/kW)', plot_height=400)
        plot.line(x='x', y='y', source=source)
        plot.title.text = "Annual Production at Varying Tilt Angles"
        plot.title.align = "center"
        plot.title.text_font = "times"
        plot.title.text_font_style = "italic"
        plot.title.text_font_size = '15pt'
        #This line is what connects the changing dropdown menu with the data that is displayed.
        select = Select(value='foo', options=['tilt_5_degrees', 'tilt_10_degrees','tilt_15_degrees',
                                             'tilt_20_degrees','tilt_25_degrees','tilt_30_degrees',
                                             'tilt_35_degrees','tilt_40_degrees','tilt_45_degrees',
                                             'tilt_50_degrees','tilt_55_degrees','tilt_60_degrees',
                                             'tilt_65_degrees','tilt_70_degrees','tilt_75_degrees',
                                             'tilt_80_degrees','tilt_85_degrees','tilt_90_degrees'])
        #javascript that actually makes the changes possible.
        select.js_on_change('value', CustomJS(args=dict(source=source, select=select), code="""
            // make a shallow copy of the current data dict
            const new_data = Object.assign({}, source.data)

            // update the y column in the new data dict from the appropriate other column
            new_data.y = source.data[select.value]

            // set the new data on source, BokehJS will pick this up automatically
            source.data = new_data
        """))

        show(column(plot, select))


In [ ]:
tilt_angle_plot_generation(location_dataframe)

Data for Ambler-Shungnak-Kobuk is being calculated
Data for Anchorage is being calculated
Data for Bethel is being calculated
Data for Chickaloon is being calculated
Data for Deering is being calculated
Data for Denali Park is being calculated
Data for Fairbanks is being calculated
Data for Fort Yukon is being calculated
